In [204]:
# coded by Chris Ha (hac541309@gmail.com) of EleutherAI, DuckAI
# for polyglot(EleutherAI) and polylingual(DuckAI) projects
# Licensed as MIT or Apache 2.0 or later versions of these licenses

In [205]:
import tokenizers
import json
import os

In [206]:
# reconstitute into dicts
# this might be just done with dict(tokenizer_json['model']['vocab'])
def recover_vocab_dict(tokenizer_json: dict) -> dict:
    tokenizer_dict = {}
    tokenizer_list = tokenizer_json['model']['vocab']
    for nlp_pair in tokenizer_list:
        tokenizer_dict[nlp_pair[0]] = nlp_pair[1]
    return tokenizer_dict

def check_duplicates(lst):
    count_dict = {}
    for item in lst:
        if item[0] in count_dict:
            count_dict[item[0]] += 1
        else:
            count_dict[item[0]] = 1
    duplicates = {key: value for key, value in count_dict.items() if value > 1}
    return duplicates

In [207]:
# equal : averages nlp
# min : takes the lowest (unlikeliest) of two options
# max : takes the highest (likeliest) of two options
nlp_mode = "max"
# this epsilon value is later used to separate byte level tokens 
# They will have the lowest negative log probs, the most unlikeliest.
# This will put them on the bottom, and other characters or tokens will be tokenized above them.

epsilon = 1E-7
# whether to bring in added_tokens of additional vocabs
with_added_tokens = False

# if not None : trim to vocab_size.
target_vocab_size = None
# the number we set the internal tokens to achieve the final intended vocab size
target_intermediate_size = None

In [208]:
# Since the ByteLevel works as its name suggests, at the byte level, it encodes each byte value to a unique visible character. 
# This means that there is a total of 256 different characters composing this alphabet.
byte_level = tokenizers.pre_tokenizers.ByteLevel(add_prefix_space = False)
byte_level.add_prefix_space = False
byte_level_alphabet = sorted(tokenizers.pre_tokenizers.ByteLevel.alphabet())
assert len(byte_level_alphabet) == 256

with open("/home/karyo/corpus/data/unicode/non_korean_exemplars.json") as f:
    non_korean_exemplars = json.load(f)
    non_korean_exemplar_byte = []
    for exemplar in non_korean_exemplars:
        non_korean_exemplar_byte.append(byte_level.pre_tokenize_str(exemplar)[0][0])

with open("/home/karyo/corpus/data/unicode/ksx-1001.txt") as f:
    korean_exemplars = f.read().split()
    korean_exemplar_byte = []
    for exemplar in korean_exemplars:
        korean_exemplar_byte.append(byte_level.pre_tokenize_str(exemplar)[0][0])

essential_byte_tokens = set(byte_level_alphabet + non_korean_exemplar_byte + korean_exemplar_byte)
essential_tokens = set(non_korean_exemplars + korean_exemplars)

In [209]:
# everything except the vocab is inherited from base
# special tokens, add tokens, preprocessing etc
base_tokenizer_path = "/home/karyo/corpus/models/hplt/merging/intermediate_c8.json"
add_tokenizer_path = "/home/karyo/corpus/models/hplt/mt_2k.json"
assert os.path.isfile(base_tokenizer_path)
assert os.path.isfile(add_tokenizer_path)
new_tokenizer_path = "/home/karyo/corpus/models/hplt/merging/final.json"
base_tokenizer = tokenizers.Tokenizer.from_file(base_tokenizer_path)

In [210]:
base_total_size = base_tokenizer.get_vocab_size(True)
base_unadded_size = base_tokenizer.get_vocab_size(False)
print(f"Total Size = {base_total_size}, Unadded size = {base_unadded_size}")

#internal representation might be different due to <bos> <eos>
base_internal_total = len(base_tokenizer.get_vocab(True))
base_internal_unadded = len(base_tokenizer.get_vocab(False))
print(f"Internal Total Size = {base_internal_total}, Unadded size = {base_internal_unadded}")
assert (base_total_size - base_unadded_size) == (base_internal_total - base_internal_unadded)
internal_offset = base_total_size - base_internal_total
added_offset = base_total_size - base_unadded_size

# we are setting the intermediate_size according to offsets
# the difference arises from how huggingface tokenizers treat added tokens and special tokens
if target_vocab_size:
    target_intermediate_size = target_vocab_size - (internal_offset + added_offset)
    print(target_intermediate_size)

Total Size = 269203, Unadded size = 269203
Internal Total Size = 269203, Unadded size = 269203


In [211]:
with open(base_tokenizer_path) as base_file,open(add_tokenizer_path) as add_file:
    base_tokenizer_json = json.load(base_file)
    add_tokenizer_json = json.load(add_file)
    # this only works for unigram
    assert base_tokenizer_json['model']['type'].lower() == "unigram"
    assert add_tokenizer_json['model']['type'].lower() == "unigram"

In [212]:
# base_tokenizer_dict = recover_vocab_dict(base_tokenizer_json)
# add_tokenizer_dict = recover_vocab_dict(add_tokenizer_json)
base_tokenizer_dict = dict(base_tokenizer_json['model']['vocab'])
add_tokenizer_dict = dict(add_tokenizer_json['model']['vocab'])
print(len(base_tokenizer_dict), len(add_tokenizer_dict))

269203 2000


In [213]:
# go through new vocab and add them
# we take the max because averaging actually reduces the negative logprob compared to other tokens
# these tokens are those that are shared! there is no reason for them to be penalized
# we take the maximum of each and bump them up a small(epsilon amount) 
# but we don't want them to exceed 0
for token in add_tokenizer_dict:
    if token in base_tokenizer_dict:
        base_tokenizer_dict[token] = min(
            0, max(base_tokenizer_dict[token], add_tokenizer_dict[token]) + epsilon
        )
    else:
        base_tokenizer_dict[token] = add_tokenizer_dict[token]

In [214]:
# sort by negative logprob, first, then length then vocab
sort_key = lambda x: [-x[1],x[0]]
# huggingface tokenizers match added/special tokens directly with the ids
# so this sorting process can cause a lot of pain
# this can be helped by adding sorted(special_tokens) before using this tool.
# another alternative is to manually realign the ids
sorted_base = sorted(base_tokenizer_dict.items(), key=sort_key,reverse=True)
print(len(sorted_base))

270177


In [215]:
# single_tokens = list(filter(lambda token:len(token[0])==1,sorted_base))
# assert len(single_tokens) == 256

In [216]:
# reconstitute vocabs. only take {target_vocab_size} number of items
# if target_vocab_size is None. the slice does nothing


# remove until size matches target
# do not remove essential tokens
if target_vocab_size:
    for pair in sorted_base:
        if len(sorted_base) <= target_vocab_size: break
        if pair[0] not in essential_byte_tokens:
            sorted_base.remove(pair)
    merged_vocab = sorted(sorted_base, key=sort_key)

# sort vocab
merged_vocab = list(map(list,sorted_base))[:target_intermediate_size]


In [217]:
# ensure byte tokens are the unlikeliest
min_log_prob = min(merged_vocab,key=lambda item:item[1])[1] - epsilon
print(len(merged_vocab), min_log_prob)

270177 -18.986092964742106


In [218]:
# uncomment to forcibly remove bytelevel token items

# merged_vocab = [item for item in merged_vocab if item[0] not in byte_level_alphabet]
# for item in merged_vocab:
#    if item[0] in byte_level_alphabet:
#        print(item)

In [219]:
# here we do 2 things
# 1. push any existing byte_level_alphabets into a low negative log prob
# 2. add any byte alphabets back into it, but not append but replace (to keep vocab_size)
not_included_count = 0
for byte in byte_level_alphabet:
    included=False
    # if byte
    for item in merged_vocab:
        if byte == item[0]:
            item[1] = min_log_prob
            included=True
            break
    # if byte token isnt included (was trimmed)
    # add it inplace of an unlikely token that is not a byte token.
    if not included:
        not_included_count +=1
        for item in reversed(merged_vocab):
            if item[0] in byte_level_alphabet:
                continue
            else:
                print(f"vocab item {item[0]}, replaced with {byte}")
                item[0] = byte
                item[1] = min_log_prob
                break
merged_vocab = sorted(merged_vocab, key=sort_key)
assert len(merged_vocab)
print(len(merged_vocab), not_included_count)

270177 0


In [220]:
print(len(base_tokenizer_json['model']['vocab']), len(merged_vocab))

269203 270177


In [221]:
base_tokenizer_json['model']['vocab'] = merged_vocab

In [222]:
# write new tokenizer
with open(new_tokenizer_path, "w") as new_file:
    json.dump(base_tokenizer_json, new_file,indent=2,ensure_ascii=False)

# check if new tokenizer can be loaded
new_tokenizer = tokenizers.Tokenizer.from_file(new_tokenizer_path)
if target_vocab_size:
    assert target_vocab_size == new_tokenizer.get_vocab_size(True)

In [223]:
exemplar_count = 0
vocab = new_tokenizer.get_vocab()
for exemplar in non_korean_exemplar_byte:
    if exemplar in vocab:
        exemplar_count += 1

korean_exemplar_count = 0
for exemplar in korean_exemplar_byte:
    if exemplar in vocab:
        korean_exemplar_count += 1

print(exemplar_count)
print(korean_exemplar_count)

4152
1084
